In [1]:
import os
os.chdir('D:\\Gnan\\DA\\KMU\\Prediction-COVID-19')
from utils import *
from model import *
from trainer import *

from tqdm.notebook import tqdm
from torch.optim.adam import Adam
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import warnings
warnings.filterwarnings('ignore')

In [2]:
path = 'Data/Variants/alpha_delta/*.csv'
dic_files = Load_files.load_files(path, -18, -16)
data = dic_files['대구']
data

,stdDay,incDec,alpha_variants,delta_variants
0,2021-04-12,16.0,4.16,0.32
1,2021-04-13,9.0,2.34,0.18
2,2021-04-14,11.0,2.86,0.22
3,2021-04-15,18.0,4.68,0.36
4,2021-04-16,4.0,1.04,0.08
...,...,...,...,...
121,2021-08-11,66.0,0.66,62.70
122,2021-08-12,39.0,0.39,37.05
123,2021-08-13,57.0,0.57,54.15
124,2021-08-14,82.0,0.82,77.90


In [12]:
path = 'Data/SIR_Origin/*.csv'
dic_files = Load_files.load_files(path, -17, -15)
data = dic_files['서울']
data

,Date,City,Susceptible,Infected,Dead,Recovered,alpha,beta,gamma
0,2020-01-20,서울,9535432,0,0,0,0.000000,0.000000,0.000000
1,2020-01-21,서울,9535432,0,0,0,0.000000,0.000000,0.000000
2,2020-01-22,서울,9535432,0,0,0,0.000000,0.000000,0.000000
3,2020-01-23,서울,9535432,0,0,0,0.000000,0.000000,0.000000
4,2020-01-24,서울,9535431,1,0,0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
1214,2023-05-20,서울,3372946,63591,6471,6092424,0.186540,0.169175,0.000031
1215,2023-05-21,서울,3368750,57027,6473,6103182,0.063087,0.028793,0.000053
1216,2023-05-22,서울,3367479,56653,6476,6104824,0.268003,0.101407,0.000018
1217,2023-05-23,서울,3362117,56269,6477,6110569,0.267742,0.099895,0.000036


In [3]:
learning_rates_list = [1e-3, 1e-4, 1e-5]
patiences_list = [20, 50]
num_layers_list = [1, 2, 4, 8]
batch_sizes_list = [32, 64]
hidden_sizes_list = [8, 16, 32]
dropout_list = [0.25]

hyperparameter_dict = {}
i = 0

for lr in learning_rates_list:
    for patience in patiences_list:
        for num_layers in num_layers_list:
            for batch_sizes in batch_sizes_list:
                for hidden_size in hidden_sizes_list:    
                    for dropout in dropout_list:
                        hyperparameter_dict[i] = [lr, patience, num_layers, batch_sizes, hidden_size, dropout]
                        i += 1
                        
print(i)

144


In [5]:
criterion = nn.MSELoss()

input_size = 3
sequence_length = 60
num_epochs = 10000

df = Prepare_df.processing(data, 'stdDay', 'alpha_variants')

x = df.iloc[:, 0:]
y = df.iloc[:,:1]

ms = MinMaxScaler()
ss = StandardScaler()

ss.fit(x)
ms.fit(y)

MinMaxScaler()

In [6]:
best_model = {}
models_list = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU']

for num_model in range(6):
    rmse_min = np.inf
    for hyperparameter in tqdm(hyperparameter_dict.values(), desc = f'대구_{models_list[num_model]}', position = 0):
        
        lr = hyperparameter[0]
        patience = hyperparameter[1]
        num_layers = hyperparameter[2]
        batch_size = hyperparameter[3]
        hidden_size = hyperparameter[4]
        dropout = hyperparameter[5]
        print('-------------------------------------------------------------------------------------')
        print('lr:', lr, ' patience:', patience, ' num_layers:', num_layers, 
              ' batch_size:', batch_size, ' hidden_size:', hidden_size, ' dropout:', dropout)
    
        x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 59, 60, 1, batch_size, 'mto')
        
        if num_model == 0:
            model = RNN(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)
            
        elif num_model == 1:
            model = LSTM(input_size = input_size,
                         hidden_size = hidden_size,
                         sequence_length = sequence_length,
                         num_layers = num_layers, 
                         dropout = dropout, 
                         device = device).to(device)
        
        elif num_model == 2:
            model = GRU(input_size = input_size,
                        hidden_size = hidden_size,
                        sequence_length = sequence_length,
                        num_layers = num_layers, 
                        dropout = dropout, 
                        device = device).to(device)
            
        elif num_model == 3:
            model = BiRNN(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        elif num_model == 4:
            model = BiLSTM(input_size = input_size,
                           hidden_size = hidden_size,
                           sequence_length = sequence_length,
                           num_layers = num_layers, 
                           dropout = dropout, 
                           device = device).to(device)

        elif num_model == 5:
            model = BiGRU(input_size = input_size,
                          hidden_size = hidden_size,
                          sequence_length = sequence_length,
                          num_layers = num_layers, 
                          dropout = dropout, 
                          device = device).to(device)

        optimizer = Adam(model.parameters(), lr = lr)
    
        loss_list, model, epoch = Trainer.Many_to_One(train_loader, 
                                                      test_loader, 
                                                      model, 
                                                      criterion, 
                                                      optimizer, 
                                                      num_epochs, 
                                                      patience,
                                                      device)
    
        label_y, predicted = predict_mto(model, df, x_ss, y_ms)
    
        if rmse_min > rmse(label_y[-7:], predicted[-7:]):
            rmse_min = rmse(label_y[-7:], predicted[-7:])
            best_model[models_list[num_model]] = [rmse_min, model, loss_list, epoch, lr, patience, 
                                                  num_layers, batch_size, hidden_size, dropout]

대구_RNN:   0%|          | 0/144 [00:00<?, ?it/s]

-------------------------------------------------------------------------------------
lr: 0.001  patience: 20  num_layers: 1  batch_size: 32  hidden_size: 8  dropout: 0.25


  0%|          | 0/10000 [00:00<?, ?it/s]


 Early Stopping / epoch: 259 loss: 0.0000
-------------------------------------------------------------------------------------
lr: 0.001  patience: 20  num_layers: 1  batch_size: 32  hidden_size: 16  dropout: 0.25


  0%|          | 0/10000 [00:00<?, ?it/s]

...


 Early Stopping / epoch: 314 loss: 0.0000
-------------------------------------------------------------------------------------
lr: 0.001  patience: 50  num_layers: 2  batch_size: 64  hidden_size: 32  dropout: 0.25


  0%|          | 0/10000 [00:00<?, ?it/s]


 Early Stopping / epoch: 359 loss: 0.0000
-------------------------------------------------------------------------------------
lr: 0.001  patience: 50  num_layers: 4  batch_size: 32  hidden_size: 8  dropout: 0.25


  0%|          | 0/10000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
best_model

{'RNN': [0.12844654679051942,
  RNN(
    (rnn): RNN(3, 32, num_layers=2, batch_first=True, dropout=0.25)
    (fc1): Linear(in_features=1920, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=64, bias=True)
    (fc3): Linear(in_features=64, out_features=1, bias=True)
    (relu): ReLU()
  ),
  [0.008096200414001942,
   0.032576289027929306,
   0.0017458695219829679,
   0.007418449502438307,
   0.006154830101877451,
   0.00318750087171793,
   0.0018498150166124105,
   0.0015694721369072795,
   0.0018393804784864187,
   0.0019276377279311419,
   0.0019807424396276474,
   0.0016840514726936817,
   0.0014858993235975504,
   0.0013448667014017701,
   0.001306244288571179,
   0.0012742681428790092,
   0.0012334729544818401,
   0.0012703239917755127,
   0.0011512563796713948,
   0.0011317331809550524,
   0.0009225658141076565,
   0.0010031572310253978,
   0.0009273126488551497,
   0.0009051239467225969,
   0.0008483328856527805,
   0.0006951811374165118,
   0.00062646

In [19]:
model_RNN = RNN(input_size = input_size,
                hidden_size = hidden_size,
                sequence_length = sequence_length,
                num_layers = num_layers, 
                dropout = dropout, 
                device = device).to(device)

model_LSTM = LSTM(input_size = input_size,
                  hidden_size = hidden_size,
                  sequence_length = sequence_length,
                  num_layers = num_layers, 
                  dropout = dropout, 
                  device = device).to(device)

model_GRU = GRU(input_size = input_size,
                hidden_size = hidden_size,
                sequence_length = sequence_length,
                num_layers = num_layers, 
                dropout = dropout, 
                device = device).to(device)

model_BiRNN = BiRNN(input_size = input_size,
                    hidden_size = hidden_size,
                    sequence_length = sequence_length,
                    num_layers = num_layers, 
                    dropout = dropout, 
                    device = device).to(device)

model_BiLSTM = BiLSTM(input_size = input_size,
                      hidden_size = hidden_size,
                      sequence_length = sequence_length,
                      num_layers = num_layers, 
                      dropout = dropout, 
                      device = device).to(device)

model_BiGRU = BiGRU(input_size = input_size,
                    hidden_size = hidden_size,
                    sequence_length = sequence_length,
                    num_layers = num_layers, 
                    dropout = dropout, 
                    device = device).to(device)

model_seq2seq_RNN = RNN_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)

model_seq2seq_LSTM = LSTM_encoder_decoder(input_size = input_size, 
                                         hidden_size = hidden_size,
                                         num_layers = num_layers, 
                                         dropout = dropout,
                                         device = device).to(device)

model_seq2seq_GRU = GRU_encoder_decoder(input_size = input_size, 
                                        hidden_size = hidden_size,
                                        num_layers = num_layers, 
                                        dropout = dropout,
                                        device = device).to(device)

model_seq2seq_BiRNN = BiRNN_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

model_seq2seq_BiLSTM = BiLSTM_encoder_decoder(input_size = input_size, 
                                              hidden_size = hidden_size,
                                              num_layers = num_layers, 
                                              dropout = dropout,
                                              device = device).to(device)

model_seq2seq_BiGRU = BiGRU_encoder_decoder(input_size = input_size, 
                                            hidden_size = hidden_size,
                                            num_layers = num_layers, 
                                            dropout = dropout,
                                            device = device).to(device)

model_dict = {}

keys = ['RNN', 'LSTM', 'GRU', 'BiRNN', 'BiLSTM', 'BiGRU', 
        'seq2seq_RNN', 'seq2seq_LSTM', 'seq2seq_GRU', 'seq2seq_BiRNN', 'seq2seq_BiLSTM', 'seq2seq_BiGRU']

models_list = [model_RNN, model_LSTM, model_GRU, 
              model_BiRNN, model_BiLSTM, model_BiGRU,
              model_seq2seq_RNN, model_seq2seq_LSTM, model_seq2seq_GRU, 
              model_seq2seq_BiRNN, model_seq2seq_BiLSTM, model_seq2seq_BiGRU]

for i, key in enumerate(keys):
    model_dict[key] = models_list[i]

In [20]:
Trained_dict = {}
num_model = 0
mape_val = 10000
for model in models_list:
    
    optimizer = Adam(model.parameters(), lr = lr)
    num_model += 1
    if num_model <= 6:
        x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 800, 60, 1, batch_size, 'mto')
        loss_list, model = Trainer.Many_to_One(train_loader, 
                                               test_loader, 
                                               model, 
                                               criterion, 
                                               optimizer, 
                                               num_epochs, 
                                               patience,
                                               device)
        label_y, predicted = predict_mto(model, df, x_ss, y_ms)
        if mape_val > mape(label_y+1, predicted):
            mape_val = vape_val = mape(label_y+1, predicted)
            Trained_dict['서울'] = [mape_val, model, data, lr, criterion, patience, num_layers, batch_size, hidden_size, dropout]
        
        
    else:
        x, y, x_ss, y_ms, train_loader, test_loader = Prepare_df.split_data(df, 800, 60, 7, batch_size, 'mtm')
        loss_list, model = Trainer.Many_to_Many(train_loader, 
                                                test_loader, 
                                                model, 
                                                criterion, 
                                                optimizer, 
                                                num_epochs, 
                                                patience,
                                                device)
        first_label_y, first_predicted = predict_mtm(model, df, x_ss, y_ms, len(df), 7, 0.5, device)
        if mape_val > mape(label_y+1, predicted):
            mape_val = vape_val = mape(label_y+1, predicted)
            Trained_dict['서울'] = [mape_val, model, data, lr, criterion, patience, num_layers, batch_size, hidden_size, dropout]
       

torch.Size([800, 60, 3])
torch.Size([800, 1])
torch.Size([357, 60, 3])
torch.Size([357, 1])

 Early Stopping / epoch: 81 loss: 0.0005
torch.Size([800, 60, 3])
torch.Size([800, 1])
torch.Size([357, 60, 3])
torch.Size([357, 1])
epoch: 100 loss: 0.0001

 Early Stopping / epoch: 121 loss: 0.0001
torch.Size([800, 60, 3])
torch.Size([800, 1])
torch.Size([357, 60, 3])
torch.Size([357, 1])
epoch: 100 loss: 0.0000

 Early Stopping / epoch: 121 loss: 0.0001
torch.Size([800, 60, 3])
torch.Size([800, 1])
torch.Size([357, 60, 3])
torch.Size([357, 1])
epoch: 100 loss: 0.0000

 Early Stopping / epoch: 121 loss: 0.0000
torch.Size([800, 60, 3])
torch.Size([800, 1])
torch.Size([357, 60, 3])
torch.Size([357, 1])
epoch: 100 loss: 0.0001

 Early Stopping / epoch: 161 loss: 0.0000
torch.Size([800, 60, 3])
torch.Size([800, 1])
torch.Size([357, 60, 3])
torch.Size([357, 1])

 Early Stopping / epoch: 61 loss: 0.0001
torch.Size([800, 60, 3])
torch.Size([800, 7, 1])
torch.Size([351, 60, 3])
torch.Size([351, 7, 1]

ValueError: cannot reshape array of size 1151 into shape (1217,1)

In [10]:
mape_val

0